# Contents
* [Starting Out](#Starting-Out)
* [Creating Runs](#Creating-Runs)
    - [`new`](#new)
    - [Using Spec Files](#Using-Spec-Files)
* [Querying Existing Runs](#Querying-Existing-Runs)
    - [`ls`](#ls)
    - [`lookup`](#lookup)
* [File IO](#File-IO)
* [Changing Runs](#Changing-Runs)
    - [`mv`](#mv)
    - [`change-description`](#change-description)
    - [`kill`](#kill)
* [Deleting Runs](#Deleting-Runs)
* [Reproducing Runs](#Reproducing-Runs)
* [Comparing Runs](#Comparing-Runs)
* [`runs-git`](#runs-git)

In [1]:
runs -y rm % && rm -f runs.db .runsrc  # start with a clean slate.

# Starting Out

We begin by creating a new `.runsrc` file and creating our database `runs.db`. Running any `runs` command will do this. Note that without `--assume-yes`, this would prompt the user for confirmation. You can also use the short flag `-y`.

In [2]:
runs --assume-yes ls

Config not found. Using default config:
{'args': '',
 'db_path': '/Users/ethan/lab-notebook/demo/runs.db',
 'dir_names': '',
 'root': '/Users/ethan/lab-notebook/demo/.runs'}



In [3]:
ls runs.db  # database that maintains metadata about runs

runs.db


In [4]:
cat .runsrc  # config file

[main]
root : /Users/ethan/lab-notebook/demo/.runs
db_path : /Users/ethan/lab-notebook/demo/runs.db
dir_names : 
args : 



The `.runsrc` establishes various defaults for the `runs` command although these can be overriden with command line arguments. We will explore some of the capabilities of this file [later](#File-IO) in the tutorial.

# Creating Runs

## `new`

In [5]:
runs -y new --path=demo --command=python --description="Demonstrate lab-notebook capabilities."

Path:
demo
Description:
Demonstrate lab-notebook capabilities.
Command sent to session:
python
List active:
tmux list-session
Attach:
tmux attach -t 'demo'



This performs the following operations:
* Checks for runs with the same path name and deletes them (after asking permission from the user if no `-y` flag).
* Creates a TMUX session with the command launched in it.
* Stores metadata about the run in `runs.db`.
* Creates directories in accordance with `.runsrc` (as we demonstrate in the [File IO](#File-IO) section of this tutorial).

In [6]:
tmux ls | grep demo

demo: 1 windows (created Wed Dec 26 13:17:00 2018) [80x24]


You can create multiple runs with a single run command:

In [7]:
runs -y new --path=demo --command="python dummy.py --flag --my-arg=1 --my-arg=2" \
            --path=demo2 --command="python dummy.py --my-arg=1" \
            --description="Demonstrate creating multiple runs at once"

Path:
demo2
Description:
Demonstrate creating multiple runs at once
Command sent to session:
python dummy.py --my-arg=1
List active:
tmux list-session
Attach:
tmux attach -t 'demo2'

Path:
demo
Description:
Demonstrate creating multiple runs at once
Command sent to session:
python dummy.py --my-arg=2 --flag --my-arg=1
List active:
tmux list-session
Attach:
tmux attach -t 'demo'



Note that we just overwrote the previous run called `demo`, meaning that we killed the associated tmux session and performed various cleanup actions. Without the `-y` flag, it would prompt the user before any change.

In [8]:
tmux ls | grep demo

demo: 1 windows (created Wed Dec 26 13:17:01 2018) [80x24]
demo2: 1 windows (created Wed Dec 26 13:17:01 2018) [80x24]


## Using Spec Files

You can also use 'spec' files to create multiple runs using cross-products of arguments. You can create a spec file by hand, or you can generate one from existing runs:

In [9]:
runs lookup command demo demo2

demo2: python dummy.py --my-arg=1
demo: python dummy.py --my-arg=2 --flag --my-arg=1


In [10]:
runs to-spec demo demo2 > run.json

`run.json` now contains a specification of the commands used in `demo` and `demo2`.

In [11]:
cat run.json

{
    "args": {
        "my-arg": [
            [
                1,
                2
            ],
            1
        ]
    },
    "command": "python dummy.py",
    "flags": [
        [
            null
        ],
        [
            "flag"
        ]
    ]
}


Note that `my-arg` contains a list of lists. This is how specs represent repeated args.

We can use this specification to generate runs that use a cross-product of the specified arguments:

In [12]:
runs -y from-spec run.json --path=from-spec-demo --description="Demonstrate the use of specs to generate runs."

Path:
from-spec-demo/0
Description:
Demonstrate the use of specs to generate runs.
Command sent to session:
python dummy.py --my-arg="2" --my-arg="1"
List active:
tmux list-session
Attach:
tmux attach -t 'from-spec-demo/0'

Path:
from-spec-demo/1
Description:
Demonstrate the use of specs to generate runs.
Command sent to session:
python dummy.py --my-arg="2" --my-arg="1" --flag
List active:
tmux list-session
Attach:
tmux attach -t 'from-spec-demo/1'

Path:
from-spec-demo/2
Description:
Demonstrate the use of specs to generate runs.
Command sent to session:
python dummy.py --my-arg="1"
List active:
tmux list-session
Attach:
tmux attach -t 'from-spec-demo/2'

Path:
from-spec-demo/3
Description:
Demonstrate the use of specs to generate runs.
Command sent to session:
python dummy.py --my-arg="1" --flag
List active:
tmux list-session
Attach:
tmux attach -t 'from-spec-demo/3'



In [13]:
tmux ls | grep demo

demo: 1 windows (created Wed Dec 26 13:17:01 2018) [80x24]
demo2: 1 windows (created Wed Dec 26 13:17:01 2018) [80x24]
from-spec-demo/0: 1 windows (created Wed Dec 26 13:17:02 2018) [80x24]
from-spec-demo/1: 1 windows (created Wed Dec 26 13:17:02 2018) [80x24]
from-spec-demo/2: 1 windows (created Wed Dec 26 13:17:02 2018) [80x24]
from-spec-demo/3: 1 windows (created Wed Dec 26 13:17:02 2018) [80x24]


Here, you can see that the generated runs use all combinations of the specified arguments:

In [14]:
runs lookup command from-spec-demo/%  # '%' is a wildcard pattern

from-spec-demo/0: python dummy.py --my-arg="2" --my-arg="1"
from-spec-demo/1: python dummy.py --my-arg="2" --my-arg="1" --flag
from-spec-demo/2: python dummy.py --my-arg="1"
from-spec-demo/3: python dummy.py --my-arg="1" --flag


# Querying Existing Runs

## `ls`

The most basic way to query runs is simply to list them:

In [15]:
runs ls %  # queries use SQL wildcard patterns ('%' matches everything)

demo2
demo
from-spec-demo/0
from-spec-demo/1
from-spec-demo/2
from-spec-demo/3


In [16]:
runs ls demo_

demo2


You can use `--active` to select only runs that have current active TMUX Sessions:

In [17]:
runs ls --active

demo
demo2
from-spec-demo/0
from-spec-demo/1
from-spec-demo/2
from-spec-demo/3


This matches the output of `tmux ls`:

In [18]:
tmux ls | grep demo

demo: 1 windows (created Wed Dec 26 13:17:01 2018) [80x24]
demo2: 1 windows (created Wed Dec 26 13:17:01 2018) [80x24]
from-spec-demo/0: 1 windows (created Wed Dec 26 13:17:02 2018) [80x24]
from-spec-demo/1: 1 windows (created Wed Dec 26 13:17:02 2018) [80x24]
from-spec-demo/2: 1 windows (created Wed Dec 26 13:17:02 2018) [80x24]
from-spec-demo/3: 1 windows (created Wed Dec 26 13:17:02 2018) [80x24]


In [19]:
runs -y kill demo

After we kill the TMUX session for the `demo` run, it no longer appears in response to `--active` queries:

In [20]:
runs ls --active

demo2
from-spec-demo/0
from-spec-demo/1
from-spec-demo/2
from-spec-demo/3


But it still shows up in our database:

In [21]:
runs ls %

demo2
demo
from-spec-demo/0
from-spec-demo/1
from-spec-demo/2
from-spec-demo/3


You can also filter by time.

In [22]:
runs ls --since $(date "+%Y-%m-%d")

demo2
demo
from-spec-demo/0
from-spec-demo/1
from-spec-demo/2
from-spec-demo/3


In [23]:
runs ls --from-last 1day

demo2
demo
from-spec-demo/0
from-spec-demo/1
from-spec-demo/2
from-spec-demo/3


For info on accepted formats, run `runs ls -h` (ommitted for brevity).

All these selection mechanisms are available to other `runs` subcommands (`rm`, `mv`, `ls`, `lookup`, `change-description`, `reproduce`, `correlate`, `kill`).

## `lookup`

You can query metadata about runs:

In [24]:
runs lookup commit from-spec-demo/%

from-spec-demo/0: ec84f3e7eece3fdab32c7c84c9d3cbcda953cc8e
from-spec-demo/1: ec84f3e7eece3fdab32c7c84c9d3cbcda953cc8e
from-spec-demo/2: ec84f3e7eece3fdab32c7c84c9d3cbcda953cc8e
from-spec-demo/3: ec84f3e7eece3fdab32c7c84c9d3cbcda953cc8e


In [25]:
runs lookup datetime from-spec-demo/%

from-spec-demo/0: 2018-12-26T13:17:02.705685
from-spec-demo/1: 2018-12-26T13:17:02.712074
from-spec-demo/2: 2018-12-26T13:17:02.718567
from-spec-demo/3: 2018-12-26T13:17:02.724969


For info about queryable fields, run `runs lookup -h` (omitted for brevity).

# File IO

In this section we will focus on two fields in the `.runsrc`:
* `dir_names` specifies directories that will be placed inside `root` and will be synchronized with run paths (created, moved, and deleted with them).
* `args` specifies flags that should be passed to every run. The `<path>` keyword gets replaced with the path of the run.

In [26]:
echo '[main]
root : /Users/ethan/lab-notebook/demo/.runs
db_path : /Users/ethan/lab-notebook/demo/runs.db
dir_names : write-dir
args : --write-path=${main:root}/write-dir/<path>/hello.txt' > .runsrc

In [27]:
cat file_io_demo.py

#! /usr/bin/env python

import argparse
from pathlib import Path

parser = argparse.ArgumentParser()
parser.add_argument('--write-path', type=Path, required=True)
args = parser.parse_args()

with args.write_path.open('w') as f:
    f.write('Hello')


When we generate multiple runs, the appropriate flags get included in the command:

In [28]:
runs -y new --path=demo1 --command='python3 file_io_demo.py' \
            --path=demo2 --command='python3 file_io_demo.py' \
            --description='Demonstrate .runsrc capabilities'

Path:
demo1
Description:
Demonstrate .runsrc capabilities
Command sent to session:
python3 file_io_demo.py --write-path=/Users/ethan/lab-notebook/demo/.runs/write-dir/demo1/hello.txt
List active:
tmux list-session
Attach:
tmux attach -t 'demo1'

Path:
demo2
Description:
Demonstrate .runsrc capabilities
Command sent to session:
python3 file_io_demo.py --write-path=/Users/ethan/lab-notebook/demo/.runs/write-dir/demo2/hello.txt
List active:
tmux list-session
Attach:
tmux attach -t 'demo2'



Note that the `--write-path` arg has been passed to each run with the value specified in `.runsrc`. Also note that the directory `.runs/write-dir` was created by the `runs` command (because of the `dir_names` section in `.runsrc`), not by `demo_script.py`:

In [29]:
sleep 1 && tree .runs/write-dir/

.runs/write-dir/
├── demo1
└── demo2

2 directories, 0 files


In subsequent sections we will see that the program keeps the `write-dir` directory in sync with any changes to a run.

# Changing Runs

## `mv`

Like the `new` command, the `mv` command not only renames commands it also
* Overwrites commands with the same name as the new name for the run.
* Renames the TMUX session.
* Updates the path name in `runs.db`.
* Moves directories listed in `.runsrc` (as described in [File IO](#File-IO) section).

Before we make any changes, let's remind ourselves of the current state of things:

In [30]:
runs ls %

demo
from-spec-demo/0
from-spec-demo/1
from-spec-demo/2
from-spec-demo/3
demo1
demo2


In [31]:
tmux ls | grep demo  # remember we killed demo, so its TMUX session no longer shows up here.

demo1: 1 windows (created Wed Dec 26 13:17:08 2018) [80x24]
demo2: 1 windows (created Wed Dec 26 13:17:08 2018) [80x24]
from-spec-demo/0: 1 windows (created Wed Dec 26 13:17:02 2018) [80x24]
from-spec-demo/1: 1 windows (created Wed Dec 26 13:17:02 2018) [80x24]
from-spec-demo/2: 1 windows (created Wed Dec 26 13:17:02 2018) [80x24]
from-spec-demo/3: 1 windows (created Wed Dec 26 13:17:02 2018) [80x24]


In [32]:
tree .runs

.runs
└── write-dir
    ├── demo1
    └── demo2

3 directories, 0 files


In [33]:
runs -y mv demo2 demo

This overwrites the run `demo`. It also moves `.runs/write-dir/demo2` and all its contents to `.runs/write-dir/demo`:

In [34]:
runs ls %  # database contains demo instead of demo2

from-spec-demo/0
from-spec-demo/1
from-spec-demo/2
from-spec-demo/3
demo1
demo


In [35]:
tree .runs  # the directory has been renamed from demo to demo2

.runs
└── write-dir
    ├── demo
    └── demo1

3 directories, 0 files


In [36]:
tmux ls | grep demo  # the TMUX session has been renamed

demo: 1 windows (created Wed Dec 26 13:17:08 2018) [80x24]
demo1: 1 windows (created Wed Dec 26 13:17:08 2018) [80x24]
from-spec-demo/0: 1 windows (created Wed Dec 26 13:17:02 2018) [80x24]
from-spec-demo/1: 1 windows (created Wed Dec 26 13:17:02 2018) [80x24]
from-spec-demo/2: 1 windows (created Wed Dec 26 13:17:02 2018) [80x24]
from-spec-demo/3: 1 windows (created Wed Dec 26 13:17:02 2018) [80x24]


## `change-description`

We can also change the description for runs

In [37]:
runs change-description from-spec-demo/% 'A new description'

In [38]:
runs lookup description --active

demo: Demonstrate .runsrc capabilities
demo1: Demonstrate .runsrc capabilities
from-spec-demo/0: A new description
from-spec-demo/1: A new description
from-spec-demo/2: A new description
from-spec-demo/3: A new description


## `kill`

As we have already seen from [earlier](#ls) in the demo, we can also kill the TMUX session for runs without deleting the database record. You can always use `tmux kill-session` for this, but `lab-notebook` allows you to kill multiple sessions at once:

In [39]:
runs -y kill from-spec-demo/%

In [40]:
tmux ls | grep demo

demo: 1 windows (created Wed Dec 26 13:17:08 2018) [80x24]
demo1: 1 windows (created Wed Dec 26 13:17:08 2018) [80x24]


In [41]:
runs ls --active

demo
demo1


In [42]:
runs ls %  # note: runs were not deleted from database

from-spec-demo/0
from-spec-demo/1
from-spec-demo/2
from-spec-demo/3
demo1
demo


# Deleting Runs

Finally let's see what happens when we delete runs.

In [43]:
runs -y rm demo

In [44]:
runs ls %

from-spec-demo/0
from-spec-demo/1
from-spec-demo/2
from-spec-demo/3
demo1


`.runs/write-dir/demo` and all its contents have been removed. This happend because `write-dir` is specified in the `dir_names` section of `.runsrc` (see the [File IO](#File-IO) section).

In [45]:
tree .runs/write-dir/

.runs/write-dir/
└── demo1

1 directory, 0 files


# Reproducing Runs

The `reproduce` command prints out commands that will exactly reproduce a queried run.

In [46]:
runs reproduce demo1

To reproduce:
git checkout ec84f3e7eece3fdab32c7c84c9d3cbcda953cc8e
runs new --path="demo1" --command="python3 file_io_demo.py " --description="Demonstrate .runsrc capabilities"


You can also reproduce multiple runs:

In [47]:
runs reproduce from-spec-demo/%

To reproduce:
git checkout ec84f3e7eece3fdab32c7c84c9d3cbcda953cc8e
runs new \
--path="from-spec-demo/0" \
--command="python dummy.py --my-arg=\"2\" --my-arg=\"1\"" \
--description="A new description" \
--path="from-spec-demo/1" \
--command="python dummy.py --my-arg=\"2\" --my-arg=\"1\" --flag" \
--description="A new description" \
--path="from-spec-demo/2" \
--command="python dummy.py --my-arg=\"1\"" \
--description="A new description" \
--path="from-spec-demo/3" \
--command="python dummy.py --my-arg=\"1\" --flag" \
--description="A new description"


You can also change the path and description of reproduced runs:

In [48]:
runs reproduce demo1 --path=new-path-name --description='Some new description.'

To reproduce:
git checkout ec84f3e7eece3fdab32c7c84c9d3cbcda953cc8e
runs new --path="new-path-name" --command="python3 file_io_demo.py --write-path=/Users/ethan/lab-notebook/demo/.runs/write-dir/demo1/hello.txt" --description="Some new description."


# Comparing Runs

In [49]:
runs lookup command from-spec-demo/0

from-spec-demo/0: python dummy.py --my-arg="2" --my-arg="1"


In [50]:
runs lookup command from-spec-demo/3

from-spec-demo/3: python dummy.py --my-arg="1" --flag


In [51]:
runs diff from-spec-demo/0 from-spec-demo/3

python dummy.py --my-arg="1" 
+ --my-arg="2" 
- --flag 


# `runs-git`

One of the most useful aspects of `lab-notebook` is the way it associates runs with the git commit that they were launched with. You can always lookup commits by hand with

In [52]:
runs lookup commit %

from-spec-demo/0: ec84f3e7eece3fdab32c7c84c9d3cbcda953cc8e
from-spec-demo/1: ec84f3e7eece3fdab32c7c84c9d3cbcda953cc8e
from-spec-demo/2: ec84f3e7eece3fdab32c7c84c9d3cbcda953cc8e
from-spec-demo/3: ec84f3e7eece3fdab32c7c84c9d3cbcda953cc8e
demo1: ec84f3e7eece3fdab32c7c84c9d3cbcda953cc8e


but `lab-notebook` includes a handy tool for working directly with path names:
```
runs-git diff demo1
runs-git checkout demo1
# etc.
```

This is equivalent to 
```
git diff $(runs lookup commit demo1 --porcelain)
git checkout $(runs lookup commit demo1 --porcelain)
# etc.
```
`runs-git` works on any `git` command that has the format 
```
git <...> <commit-hash>
```